Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Предлагаю идти по следующему плану:

    

**1.**

        1.1 загрузка, знакомство с данными
        1.2 лемматизация
        1.3 разбиение данных на выборки: train, valid, test.
        1.4 преобразование выборок в матрицы tf_idf
        
**2.**

        2.1 подбор параметров, обучение моделей и тестировоание на валидационой выборке
        2.2 выбор лучших из обученных моделей и тестирование на тестовой выборке

**3**

        3.1 финальный вывод

**Набор инструментов:**

In [1]:
import pandas as pd
from scipy import stats as st
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_curve, roc_auc_score, accuracy_score


In [2]:
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import torch
import transformers
from tqdm import notebook
from sklearn.model_selection import cross_val_score

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1. Подготовка

Загрузим и ознакомимся с данными

In [5]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


пропусков нет.

In [7]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Судя по всему язык комментариев - английский.

In [8]:
len_twit = 0
ind = 0
for i in range(len(df)):
    if len(df['text'].loc[i]) > len_twit:
        len_twit = len(df['text'].loc[i])
        ind = i

In [9]:
len_twit

5000

5000 символов - как небольшой рассказ. Я посмотрел его - повторяющиеся спамерские фразы, аналогично и с комментариями в 4997, 4994. 

Комментарии представляют собой спам в виде повторяющихся фраз или слов. Я бы дополнил модель машинного обучения функцией, которая прежде, чем отправить данные на анализ модели, смотрела есть ли повторы в комментариях - можно ли его разделить на несколько идентичных фрагментов, если да - даже не отправлять на модерацию, а сразу удалять.

Посмотрим насколько сбалансированы классы:

In [10]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Классы несбалансированные. Имея опыт работы по прошлым проектам - лучший результат, если работать с решающим деревом или случайным лесом - получается, если модель учится на несбалансированной выборке, но с изменением веса класса.

Лемматизация и токенизация

в сети прочитал, что pymystem не самый оптимальный инструмент. Давайте посмотрим на время выполнения однотипной процедуры разными лемматизаторами:

In [11]:
m = Mystem()
aa = m.lemmatize('Caring')

In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
%%time
print(m.lemmatize('Caring'))

['Caring', '\n']
CPU times: user 680 µs, sys: 64 µs, total: 744 µs
Wall time: 15.9 ms


In [14]:
%%time
print(lemmatizer.lemmatize("caring"))

caring
CPU times: user 3.07 s, sys: 183 ms, total: 3.25 s
Wall time: 3.36 s


выбор очевиден! 

Далее работаем с лемматизатором nltk



Перед лемматизацией нужно очистить текст от ненужных символов.

Для этого напишем функции:

    1. для лемматизации

In [15]:
def lemmatize(text):
    m = WordNetLemmatizer()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list) 
        
    return(lemm_text)

    2. для очистки текста

In [16]:
def clear_text(text):
    re_list = re.sub(r"[^a-zA-Z']", ' ', text)
    re_list = re_list.split()
    re_list = " ".join(re_list)
    return(re_list)

Лемматизация

Старший товарищ (наш наставник помощник, консультант и мастер в Slack) сообщил, что для Puthon 3 приводить текст в юникод нет надобности. (если я верно понял: " у нас тут третий питончик, в юникод ничего преобразовывать не надо.", хотя в тренажере мы преобразовывали)

In [17]:
corpus = list(df['text'].apply(lambda x: lemmatize(clear_text(x))))

Мы подготовили наш текст для анализа:
    
    очистили его от ненужных символов
    лемматизировали

теперь разделим на тренировочную, валидационную и тестовую выборки и подготовим матрицу tf_idf.



Выделим признаки - (наш текст) и целевой признак - df['toxic']

In [18]:
target = df['toxic'].values

In [19]:
x_train, x_val, y_train, y_val = train_test_split(corpus, target, test_size=0.4, random_state=12)

In [20]:
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=12)

Получив выборки, подготовим матрицы tf_idf

In [21]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


обучим на тренировочной выборке и преобразуем наши выборки.

In [22]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

tf_idf_maker = count_tf_idf.fit(x_train)

In [23]:
x_train = tf_idf_maker.transform(x_train)
x_val = tf_idf_maker.transform(x_val)
x_test = tf_idf_maker.transform(x_test)

**Вывод по шагу**

Посмотрели на данные - без пропусков.

Классы не сбалансирвоаны, но попробуем поработать с несбалансированными выборками, изменяя вес класса.

Познакомились с примерами комментариев.

Подготовили тренировочную, валидационную и тестовую выборки в виде матрицы tf_idf.

# 2. Обучение

В этом шаге обучим следующие моделиЖ

    1. Логистическаая регрессия
    2. Дерево решений
    3. Случайный лес
    4. Cat-boosting

Основная метрика - f1, но нужно заметить, что токсических комментариев меньше, но пропускать их категорически нельзя. Я считаю, что тут очень важны precision - как долz объектов, названных классификатором положительными и при этом действительно являющимися положительными, а recall как доля объектов положительного класса из всех объектов положительного класса.

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
model_log_reg2 = LogisticRegression(class_weight = 'balanced', random_state=12)
model_log_reg2.fit(x_train, y_train)
predicted_valid = model_log_reg2.predict_proba(x_val)

probabilities_one_valid = predicted_valid[:, 1]

f1_max = 0
threshold_max = 0
for threshold in np.arange(0, 0.9, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(y_val, predicted_valid)
    if f1_max < f1:
        f1_max = f1
        threshold_max = threshold
        print('threshould:', threshold_max)
        print('recall:', recall_score(y_val, predicted_valid))
        print('precision:', precision_score(y_val, predicted_valid))
        print('f1:', f1_score(y_val, predicted_valid))
        print('AUC-ROC', roc_auc_score(y_val, predicted_valid))
        print()
        print()

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


threshould: 0.0
recall: 1.0
precision: 0.10064548474023939
f1: 0.18288447303991345
AUC-ROC 0.5


threshould: 0.02
recall: 0.9996886674968867
precision: 0.11667453944260746
f1: 0.20896105163830406
AUC-ROC 0.5763546814594043


threshould: 0.04
recall: 0.99906600249066
precision: 0.1377371448192978
f1: 0.24209732176537158
AUC-ROC 0.6495748101785054


threshould: 0.06
recall: 0.9962640099626401
precision: 0.16103869961250064
f1: 0.27726032144868523
AUC-ROC 0.7077167029117779


threshould: 0.08
recall: 0.9940846824408468
precision: 0.18632199334772714
f1: 0.3138237751240847
AUC-ROC 0.7541324394714164


threshould: 0.1
recall: 0.9897260273972602
precision: 0.2114962411017231
f1: 0.3485172394891191
AUC-ROC 0.7883965653675034


threshould: 0.12
recall: 0.9850560398505604
precision: 0.23798420458819106
f1: 0.38335251711395163
AUC-ROC 0.8160420607586716


threshould: 0.14
recall: 0.9797633872976339
precision: 0.26445378151260507
f1: 0.41649020645844365
AUC-ROC 0.8374010302804106


threshould: 0.

**f1** мера на валидационной выборке при **пороге = 0,64** составила **0,78**

**recall** мера на валидационной выборке при **пороге = 0,64** составила **0,76**

**precision** мера на валидационной выборке при **пороге = 0,64** составила **0,79**

таким образом логистическая регрессия проходит в следующий шаг - шаг тестирования, причем не плохой результат за сравнительно малое время.

т.к. мы изменили вес класса, напишем функцию для получения прогноза от логистической регрессии:

In [26]:
def get_log_reg_pred(data):
    predicted_valid = model_log_reg2.predict_proba(data)
    probabilities_one_valid = predicted_valid[:, 1]
    predicted_valid = probabilities_one_valid > 0.64
    return(predicted_valid)

        Решающее Дерево

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
f1_max = 0
threshold_max = 0
depth_max = 0
for depth in range(1, 21, 1):
    model = DecisionTreeClassifier(random_state = 12345, max_depth=depth)
    model.fit(x_train, y_train)
    
    probabilities_valid = model.predict_proba(x_val)
    probabilities_one_valid = probabilities_valid[:, 1]
    
    for threshold in np.arange(0, 0.9, 0.02):
        predicted_valid = probabilities_one_valid > threshold
        f1 = f1_score(y_val, predicted_valid)
        if f1_max < f1:
            f1_max = f1
            threshold_max = threshold
            depth_max = depth
            precision = precision_score(y_val, predicted_valid)
            recall = recall_score(y_val, predicted_valid)
            f1 = f1_score(y_val, predicted_valid)
            auc_roc = roc_auc_score(y_val, predicted_valid)
            print("Глубина = {:.0f}, Порог = {:.2f} | Precision = {:.3f},Recall = {:.3f}, f1 = {:.3f}, AUC-ROC = {:.3f}".format(depth_max, threshold_max, precision, recall, f1, auc_roc))
            print()

Глубина = 1, Порог = 0.00 | Precision = 0.101,Recall = 1.000, f1 = 0.183, AUC-ROC = 0.500

Глубина = 1, Порог = 0.10 | Precision = 0.962,Recall = 0.152, f1 = 0.262, AUC-ROC = 0.575

Глубина = 2, Порог = 0.10 | Precision = 0.959,Recall = 0.227, f1 = 0.367, AUC-ROC = 0.613

Глубина = 3, Порог = 0.08 | Precision = 0.948,Recall = 0.273, f1 = 0.424, AUC-ROC = 0.636

Глубина = 4, Порог = 0.08 | Precision = 0.945,Recall = 0.300, f1 = 0.456, AUC-ROC = 0.649

Глубина = 5, Порог = 0.08 | Precision = 0.937,Recall = 0.339, f1 = 0.498, AUC-ROC = 0.668

Глубина = 6, Порог = 0.08 | Precision = 0.902,Recall = 0.376, f1 = 0.531, AUC-ROC = 0.686

Глубина = 7, Порог = 0.08 | Precision = 0.897,Recall = 0.399, f1 = 0.552, AUC-ROC = 0.697

Глубина = 8, Порог = 0.08 | Precision = 0.902,Recall = 0.417, f1 = 0.571, AUC-ROC = 0.706

Глубина = 9, Порог = 0.08 | Precision = 0.904,Recall = 0.430, f1 = 0.583, AUC-ROC = 0.712

Глубина = 10, Порог = 0.06 | Precision = 0.908,Recall = 0.438, f1 = 0.591, AUC-ROC = 0.717

In [29]:
param_grid = {'max_depth' : [10, 17, 20]}

dt_mod = DecisionTreeClassifier(random_state=12)

In [30]:
dt_grid = GridSearchCV(estimator=dt_mod, param_grid=param_grid, cv=5)

dt_grid.fit(x_train, y_train)

dt_grid.best_params_

{'max_depth': 20}

In [31]:
f1_max=0
for threshold in np.arange(0, 0.9, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(y_val, predicted_valid)
    if f1_max < f1:
        f1_max = f1
        threshold_max = threshold
        precision = precision_score(y_val, predicted_valid)
        recall = recall_score(y_val, predicted_valid)
        f1 = f1_score(y_val, predicted_valid)
        auc_roc = roc_auc_score(y_val, predicted_valid)
        print("Глубина = {:.0f}, Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}, f1 = {:.3f}, AUC-ROC = {:.3f}".format(depth_max, threshold_max, precision, recall, f1, auc_roc))
        print()

Глубина = 20, Порог = 0.00 | Точность = 0.098, Полнота = 0.973, f1 = 0.179, AUC-ROC = 0.488

Глубина = 20, Порог = 0.06 | Точность = 0.873, Полнота = 0.525, f1 = 0.655, AUC-ROC = 0.758



Решающее дерево дало результат хуже, чем логистическая регрессия, f1 = 0,655, оставим дерево на этом шаге.

Посмотрим что даст нам лес.

        Random Forest Classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
param_grid = {'n_estimators': [50, 100, 150],
    'max_depth' : [10, 17, 20]}

rf_mod = RandomForestClassifier(random_state=12)


In [34]:
rf_grid = GridSearchCV(estimator=rf_mod, param_grid=param_grid, cv=3) # в документации советуют брать cv минимум 5, но тогда очень долго думает

rf_grid.fit(x_train, y_train)

rf_grid.best_params_

{'max_depth': 17, 'n_estimators': 50}

In [35]:
f1_max = 0
probabilities_valid = rf_grid.predict_proba(x_val)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.1, 0.9, 0.1):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(y_val, predicted_valid)
    if f1_max < f1:
        f1_max = f1
        threshold_max = threshold
        print('F1 ', f1_max)
        print('порог', threshold_max)
        print()
        print()

F1  0.39382845188284515
порог 0.1




/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
for threshold in np.arange(0.1, 0.9, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    f1 = f1_score(y_val, predicted_valid)
    if f1_max < f1:
        f1_max = f1
        threshold_max = threshold
        print('F1 ', f1_max)
        print('порог', threshold_max)
        print()

F1  0.6583771311370337
порог 0.11

F1  0.7061774851050486
порог 0.12



Случайный лес дал результат лучший, чем решающее дерево, но не превысил результат логистической регрессии.

        CatBoostClassifier

In [37]:
from catboost import CatBoostClassifier

In [38]:
cb_model = CatBoostClassifier(eval_metric="F1",
                                   iterations=500, 
                                   max_depth=5, 
                                   learning_rate=0.3, 
                                   random_state=12,
                             verbose=10)

In [39]:
cb_model.fit(
   x_train, y_train,
   eval_set=(x_val, y_val))

0:	learn: 0.3632715	test: 0.3640495	best: 0.3640495 (0)	total: 3.38s	remaining: 28m 4s
10:	learn: 0.5501500	test: 0.5604759	best: 0.5608953 (8)	total: 31s	remaining: 22m 56s
20:	learn: 0.5932431	test: 0.6105708	best: 0.6105708 (20)	total: 58.5s	remaining: 22m 13s
30:	learn: 0.6366673	test: 0.6443035	best: 0.6443035 (30)	total: 1m 26s	remaining: 21m 50s
40:	learn: 0.6608035	test: 0.6634557	best: 0.6634557 (40)	total: 1m 54s	remaining: 21m 26s
50:	learn: 0.6789953	test: 0.6779929	best: 0.6779929 (50)	total: 2m 23s	remaining: 21m 4s
60:	learn: 0.6911287	test: 0.6866549	best: 0.6866549 (60)	total: 2m 52s	remaining: 20m 39s
70:	learn: 0.7029144	test: 0.6983759	best: 0.6986460 (69)	total: 3m 20s	remaining: 20m 10s
80:	learn: 0.7124562	test: 0.7071442	best: 0.7081340 (78)	total: 3m 48s	remaining: 19m 42s
90:	learn: 0.7207800	test: 0.7145032	best: 0.7145032 (89)	total: 4m 17s	remaining: 19m 16s
100:	learn: 0.7290213	test: 0.7172597	best: 0.7172597 (100)	total: 4m 45s	remaining: 18m 49s
110:	le

Посмотрим результат при тестировании.

### Тестирование

        Логистическая регрессия

In [40]:
print('recall:', recall_score(y_test, get_log_reg_pred(x_test)))
print('precision:', precision_score(y_test, get_log_reg_pred(x_test)))
print('f1:', f1_score(y_test, get_log_reg_pred(x_test)))
print('AUC-ROC', roc_auc_score(y_test, get_log_reg_pred(x_test)))

recall: 0.7764240024367957
precision: 0.7790342298288508
f1: 0.7777269260106788
AUC-ROC 0.8755862677733014


сохраним значения в переменных:

In [41]:
recall_lr = 0.776
precision_lr =  0.779
f1_lr =  0.777

        Cat-Boosting

In [42]:
print('recall:', recall_score(y_test, cb_model.predict(x_test)))
print('precision:', precision_score(y_test, cb_model.predict(x_test)))
print('f1:', f1_score(y_test, cb_model.predict(x_test)))
print('AUC-ROC', roc_auc_score(y_test, cb_model.predict(x_test)))

recall: 0.6612854096862626
precision: 0.8974782968168665
f1: 0.7614871974745703
AUC-ROC 0.8263118861786999


In [47]:
recall_cb = 0.661
precision_cb = 0.897
f1_cb = 0.761
AUC_ROC_cb = 0.826

CatBoost показал результат чуть хуже линейной регрессии **f1** = **0.76**,
но precision больше, то есть ложно положительных меньше, но зато больше ложно отрицательных.

Чтобы было нагляднее, давайте посмотрим на confusion_matrix:

**логистическая регрессия**

In [43]:
confusion_matrix(y_test, get_log_reg_pred(x_test))

array([[27909,   723],
       [  734,  2549]])

**cat_boost**

In [44]:
confusion_matrix(y_test, cb_model.predict(x_test))

array([[28384,   248],
       [ 1112,  2171]])

Логистическаая регрессия меньше пропускает токсичные комментарии, что для нас крайне важно. И учится и работает быстрее.

# 3. Выводы

Мы обучили в данном проекте 4 модели:
    
        1. LogisticClassifier
        2. CatBoostClassifier
        3. RandomForestClassifier
        4. DesicionTreeClassifier
        
Основное требование - f1_score больше 0,75.
Логистическая регрессия показала 0,777, кэт-бустинг 0,76, но!

Логистическая регрессия меньше пропускает токсических комментариев, быстрее учится и работает, менее требовательна к системным ресурсам.

Поэтому из сделаных мною моделей - логистическая регрессия подходит лучше.

В этой работе я не стал писать дополнительных функций, но если бы пришлось дополнить модель, если бы понадобилась функция, которая разом уберет комментарии, которые может даже и не велеки по количеству слов или символов, хотя они все максимально длинные.

То я бы пошел по следующему алгоритму:

    преобразовал комментарий в мешок слов. 
    проверил бы соответствует такой вектор условиям:
            
            - короткий вектор (сейчас не буду определять число,т.к. не проводил аналитику в данном направлении)
            - большинство значений одинаково, например [10,10,9]
    
    если соответствует - сразу удаляем, не напрягая модератора и модель.